# When you have a bipolar pair that is CA1 or DG, how often is each electrode in each region? Or white matter?

In [3]:
import pandas as pd; pd.set_option('display.max_columns', 30)
import numpy as np
from cmlreaders import CMLReader, get_data_index
from ptsa.data.filters import ButterworthFilter
import sys
import os
import matplotlib.pyplot as plt
%matplotlib inline
from pylab import *
from copy import copy
from scipy import stats
import pickle
plt.rcParams['pdf.fonttype'] = 42; plt.rcParams['ps.fonttype'] = 42 # fix fonts for Illustrator
sys.path.append('/home1/john/Downloads/ptsa_plot-master/')
sys.path.append('/home1/john/johnModules')
from brain_labels import HPC_labels, ENT_labels, PHC_labels, temporal_lobe_labels,\
                         MFG_labels, IFG_labels, nonHPC_MTL_labels
%load_ext autoreload
%autoreload
from general import *
from SWRmodule import *

df = get_data_index("r1") # all RAM subjects
exp = 'FR1' # 'FR1' 'catFR1' 'RepFR1'

/home1/john/anaconda3/envs/env1/lib/python3.7/site-packages/ptsa/data/timeseries.py:526: FutureWarning: xarray subclass TimeSeriesX should explicitly define __slots__
  class TimeSeriesX(TimeSeries):


In [22]:
# 575 FR sessions. first 18 of don't load so skip those 
exp_df = df[df.experiment==exp]
if exp == 'FR1':
    exp_df = exp_df[
                    ((df.subject!='R1015J') | (df.session!=0)) & 
                    ((df.subject!='R1063C') | (df.session!=1)) & 
                    ((df.subject!='R1093J') | (~df.session.isin([1,2]))) &
                    ((df.subject!='R1100D') | (~df.session.isin([0,1,2]))) &
                    ((df.subject!='R1120E') | (df.session!=0)) &
                    ((df.subject!='R1122E') | (df.session!=2)) &
                    ((df.subject!='R1154D') | (df.session!=0)) &
                    ((df.subject!='R1186P') | (df.session!=0)) &
                    ((df.subject!='R1201P') | (~df.session.isin([0,1]))) &
                    ((df.subject!='R1216E') | (~df.session.isin([0,1,2]))) &
                    ((df.subject!='R1277J') | (df.session!=0)) &
                    ((df.subject!='R1413D') | (df.session!=0)) & 
                    ((df.subject!='R1123C') | (df.session!=2)) & # artifacts that bleed through channels (see SWR FR1 prob sessions ppt)
                    ((df.subject!='R1151E') | (~df.session.isin([1,2]))) & # more bleed-through artifacts (see same ppt)
                    ((df.subject!='R1275D') | (df.session!=3))  # 3rd session an actual repeat of 2nd session (Paul should have removed from database by now)
    #                 (df.subject!='R1065J') # sub with 9000 trials
                   ] 
elif exp == 'catFR1': 
    exp_df = exp_df[
                    ((df.subject!='R1044J') | (df.session!=0)) & # too few trials to do pg pairwise corr
                    ((df.subject!='R1491T') | (~df.session.isin([1,3,5]))) & # too few trials to do pg pairwise corr
                    ((df.subject!='R1486J') | (~df.session.isin([4,5,6,7]))) & # repeated data...will be removed at some point... @@
                    ((df.subject!='R1501J') | (~df.session.isin([0,1,2,3,4,5]))) & # these weren't catFR1 (and they don't load right anyway)
                    ((df.subject!='R1235E') | (df.session!=0)) & # split EEG filenames error...documented on Asana
                    ((df.subject!='R1310J') | (df.session!=1)) & # session 1 is just a repeat of session 0
                    ((df.subject!='R1239E') | (df.session!=0)) # some correlated noise (can see in catFR1 problem sessions ppt)
    ]
elif exp == 'RepFR1':
    exp_df = exp_df[
                    (df.subject!='R1564J') # clearly something wrong with these EEG when looking at ripple raster
                    ]
exp_df.shape

# exp_df = exp_df[540:] # for FR1 this is R1385E-onwwards

exp_df

/home1/john/anaconda3/envs/env1/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


(565, 17)

,Recognition,all_events,contacts,experiment,import_type,localization,math_events,montage,original_experiment,original_session,pairs,ps4_events,session,subject,subject_alias,system_version,task_events
0,NaN,protocols/r1/subjects/R1001P/experiments/FR1/s...,protocols/r1/subjects/R1001P/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1001P/experiments/FR1/s...,0,NaN,0,protocols/r1/subjects/R1001P/localizations/0/m...,NaN,0,R1001P,R1001P,NaN,protocols/r1/subjects/R1001P/experiments/FR1/s...
1,NaN,protocols/r1/subjects/R1001P/experiments/FR1/s...,protocols/r1/subjects/R1001P/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1001P/experiments/FR1/s...,0,NaN,1,protocols/r1/subjects/R1001P/localizations/0/m...,NaN,1,R1001P,R1001P,NaN,protocols/r1/subjects/R1001P/experiments/FR1/s...
9,NaN,protocols/r1/subjects/R1002P/experiments/FR1/s...,protocols/r1/subjects/R1002P/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1002P/experiments/FR1/s...,0,NaN,0,protocols/r1/subjects/R1002P/localizations/0/m...,NaN,0,R1002P,R1002P,NaN,protocols/r1/subjects/R1002P/experiments/FR1/s...
10,NaN,protocols/r1/subjects/R1002P/experiments/FR1/s...,protocols/r1/subjects/R1002P/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1002P/experiments/FR1/s...,0,NaN,1,protocols/r1/subjects/R1002P/localizations/0/m...,NaN,1,R1002P,R1002P,NaN,protocols/r1/subjects/R1002P/experiments/FR1/s...
17,NaN,protocols/r1/subjects/R1003P/experiments/FR1/s...,protocols/r1/subjects/R1003P/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1003P/experiments/FR1/s...,0,NaN,0,protocols/r1/subjects/R1003P/localizations/0/m...,NaN,0,R1003P,R1003P,1.0,protocols/r1/subjects/R1003P/experiments/FR1/s...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2530,NaN,protocols/r1/subjects/R1565T/experiments/FR1/s...,protocols/r1/subjects/R1565T/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1565T/experiments/FR1/s...,0,NaN,NaN,protocols/r1/subjects/R1565T/localizations/0/m...,NaN,0,R1565T,R1565T,1.0,protocols/r1/subjects/R1565T/experiments/FR1/s...
2540,NaN,protocols/r1/subjects/R1569T/experiments/FR1/s...,protocols/r1/subjects/R1569T/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1569T/experiments/FR1/s...,0,NaN,NaN,protocols/r1/subjects/R1569T/localizations/0/m...,NaN,0,R1569T,R1569T,1.0,protocols/r1/subjects/R1569T/experiments/FR1/s...
2545,NaN,protocols/r1/subjects/R1571T/experiments/FR1/s...,protocols/r1/subjects/R1571T/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1571T/experiments/FR1/s...,0,NaN,NaN,protocols/r1/subjects/R1571T/localizations/0/m...,NaN,0,R1571T,R1571T,1.0,protocols/r1/subjects/R1571T/experiments/FR1/s...
2549,NaN,protocols/r1/subjects/R1572T/experiments/FR1/s...,protocols/r1/subjects/R1572T/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1572T/experiments/FR1/s...,0,NaN,NaN,protocols/r1/subjects/R1572T/localizations/0/m...,NaN,0,R1572T,R1572T,1.0,protocols/r1/subjects/R1572T/experiments/FR1/s...


In [23]:
# load df from hamming run so can take only trials that loaded from it

hamming_df = pd.read_csv('/home1/john/SWR/compiled/hamming_df.csv') 
hamming_df = hamming_df.drop_duplicates().reset_index(drop=True)
# change session value to int of the session number
for tr in range(len(hamming_df)):
    hamming_df.at[tr,'session'] = int(hamming_df.iloc[tr].session[7:])
hamming_df

,subject,session,channel_num
0,R1001P,0,21
1,R1001P,0,22
2,R1001P,0,57
3,R1001P,1,21
4,R1001P,1,22
...,...,...,...
1466,R1467M,0,16
1467,R1467M,0,17
1468,R1467M,0,18
1469,R1467M,0,19


In [36]:
def getBestRegionNames(contact_pairs,contact_localization):

    # go through this manual list order and select the region from the best atlas available
    # details on list here: https://memory-int.psych.upenn.edu/InternalWiki/index.php/RAM_data
    
    # input: row of df from pairs and row of df from localization (or [] if localization doesn't exist)
    # output: the region and atlas for this contact (single electrode)
    
    region = []
    atlas_type = []
    region_reached = 0

    if 'stein.region' in contact_pairs: # if 'stein' in contact_row.dtype.names:
        test_region = str(contact_pairs['stein.region'].values[0])
        if (test_region is not None) and (len(test_region)>1) and \
           (test_region not in 'None') and (test_region != 'nan') and (region_reached==0):
            region = test_region.lower()
            atlas_type = 'stein'
            region_reached = 1

    if 'das.region' in contact_pairs:
        test_region = str(contact_pairs['stein.region'].values[0])
        if (test_region is not None) and (len(test_region)>1) and \
           (test_region not in 'None') and (test_region != 'nan') and (region_reached==0):
            region = test_region.lower()
            atlas_type = 'das'
            region_reached = 1
    if 'atlases.mtl' in contact_localization:    
        test_region = str(contact_localization['atlases.mtl'].values[0])    
        if (test_region is not None) and (len(test_region)>1) and \
           (test_region not in 'None') and (test_region != 'nan') and (region_reached==0):
            region = test_region.lower()
            atlas_type = 'MTL_localization'
            region_reached = 1
            
    if 'atlases.whole_brain' in contact_localization:
        test_region = str(contact_localization['atlases.whole_brain'].values[0])
        if (test_region is not None) and (len(test_region)>1) and \
           (test_region not in 'None') and (test_region != 'nan') and (region_reached==0):
            region = test_region.lower()
            atlas_type = 'wb_localization' 
            region_reached = 1
            
    if 'wb.region' in contact_pairs:
        test_region = str(contact_pairs['wb.region'].values[0])
        if (test_region is not None) and (len(test_region)>1) and \
           (test_region not in 'None') and (test_region != 'nan') and (region_reached==0):
            region = test_region.lower()
            atlas_type = 'wb'  
            region_reached = 1
            
    if 'atlases.dk' in contact_localization:
        test_region = str(contact_localization['atlases.dk'].values[0])
        if (test_region is not None) and (len(test_region)>1) and \
           (test_region not in 'None') and (test_region != 'nan') and (region_reached==0):
            region = test_region.lower()
            atlas_type = 'dk_localization'  
            region_reached = 1

    if 'dk.region' in contact_pairs:
        test_region = str(contact_pairs['dk.region'].values[0])
        if (test_region is not None) and (len(test_region)>1) and \
           (test_region not in 'None') and (test_region != 'nan') and (region_reached==0):
            region = test_region.lower()
            atlas_type = 'dk'   
            region_reached = 1

    if 'ind.corrected.region' in contact_pairs:
        test_region = str(contact_pairs['ind.corrected.region'].values[0])
        if (test_region is not None) and (len(test_region)>1) and \
           (test_region not in 'None') and (test_region != 'nan') and (region_reached==0):
            region = test_region.lower()
            atlas_type = 'ind.corrected' 
            region_reached = 1
            
    if 'ind.region' in contact_pairs:
        test_region = str(contact_pairs['ind.region'].values[0])
        if (test_region is not None) and (len(test_region)>1) and \
           (test_region not in 'None') and (test_region != 'nan') and (region_reached==0):
            region = test_region.lower()
            atlas_type = 'ind'  
            region_reached = 1
    elif len(region) == 0:
        region = 'No atlas'
        atlas_type = 'No atlas'
                
    return region, atlas_type

In [37]:
ca1_contacts = []
ca1_atlases = []
dg_contacts = []
dg_atlases = []

for row in exp_df.itertuples(): #sub_df.itertuples():   
    
    if sum( (row.subject==hamming_df.subject) & (row.session==hamming_df.session) )>0: # if this session is kept
        
        # get the HPC channels to target
        analyzed_channels = hamming_df[(row.subject==hamming_df.subject) & (row.session==hamming_df.session)].channel_num.values
    
        sub = row.subject; session = row.session; exp = row.experiment
        mont = int(row.montage); loc = int(row.localization)
        reader = CMLReadDFRow(row)
            
        pairs = reader.load('pairs')
        try:
            localizations = reader.load('localization')
#             localization_exists = 1
        except:
            localizations = []
        
        tal_struct, bipolar_pairs, mpchans = get_bp_tal_struct(sub, montage=mont, localization=loc)
        elec_regions,atlas_type,pair_number,has_stein_das = get_elec_regions(localizations,pairs)

        # for each pair, get the two contacts, and then figure out what their region is. Then save both in a list        
        contacts = reader.load('contacts')
        
        for channel in analyzed_channels:

            pair_label = pairs.iloc[channel].label
            split_pair = pair_label.split('-')
            
            region_pair = []
            atlas_pair = []
            contact_localization = []
            for contact in split_pair:
                contact_pairs = contacts.iloc[findInd(contacts.label==contact)]
                if len(localizations)>0:
                    contact_localization = localizations.iloc[findInd(localizations.iloc[:,0]['contacts'].index.values==contact)]

                region,atlas = getBestRegionNames(contact_pairs,contact_localization)

                region_pair.append(region) # get the pair of elecs for this bipolar pair
                atlas_pair.append(atlas)
        
                            
            if elec_regions[channel].find('ca1')>-1:
                ca1_contacts.append(region_pair)
                ca1_atlases.append(atlas_pair) 
            elif elec_regions[channel].find('dg')>-1:
                dg_contacts.append(region_pair)
                dg_atlases.append(atlas_pair)                
                


/home1/john/Downloads/cmlreaders-master/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files found: /protocols/r1/subjects/R1001P/localizations/0/montages/0/neuroradiology/current_processed/pairs.json
/data/eeg/R1001P/tal/R1001P_talLocs_database_bipol.mat returning the first file found
  warnings.warn(msg, MultiplePathsFoundWarning)
/home1/john/anaconda3/envs/env1/lib/python3.7/site-packages/ptsa/data/readers/tal.py:43: FutureWarning: Lab-specific readers may be moved to the cmlreaders package (https://github.com/pennmem/cmlreaders)
  FutureWarning)
/home1/john/Downloads/cmlreaders-master/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files found: /protocols/r1/subjects/R1001P/localizations/0/montages/0/neuroradiology/current_processed/contacts.json
/data/eeg/R1001P/tal/R1001P_talLocs_database_monopol.mat returning the first file found
  warnings.warn(msg, MultiplePathsFoundWarning)
/home1/john/Downloads/cmlreaders-master/cmlreaders/path_finder.p

KeyboardInterrupt: 

In [39]:
ca1_contacts
dg_contacts

[['left sub', 'left dg'],
 ['left dg', 'left ca1'],
 ['left sub', 'left dg'],
 ['left dg', 'left ca1'],
 ['right ca1', 'right ca1'],
 ['left ca1', 'left ca1'],
 ['right ca1', 'right ca1'],
 ['right ca1', []],
 ['left ca1', 'left ca1'],
 ['right ca1', 'right ca1'],
 ['right ca1', []],
 ['left ca1', 'left ca1'],
 [[], 'left ca1'],
 ['left ca1', 'left ca1'],
 [[], 'left ca1'],
 ['right ca1', 'right hippocampus'],
 ['right ca1', 'right hippocampus'],
 ['left ca1', 'left ca1'],
 ['left dg', []],
 ['left ca1', 'left ca1'],
 ['left dg', []],
 ['right ca1', 'right dg'],
 ['right ca1', 'right ca1'],
 ['left ca1', 'left cerebral white matter'],
 ['left ca1', 'left ca1'],
 ['left ca1', 'left ca1'],
 ['left ca1', 'left dg'],
 ['left sub', 'left ca1'],
 ['right phg parahippocampal gyrus', 'right ca1'],
 ['right ca1', 'right ca1'],
 ['right ca1', 'right ca1'],
 ['right ca1', 'right ca1'],
 ['right ca1', 'right ca1'],
 ['right ca1', 'right lateral ventricle'],
 ['left amy', 'left dg'],
 ['left ca1', 

[['right dg', 'right ca1'],
 ['right dg', 'right ca1'],
 ['right sub', 'right dg'],
 ['right dg', 'right ca1'],
 ['right sub', 'right dg'],
 ['right dg', 'right ca1'],
 ['left dg', 'left dg'],
 ['left dg', 'left dg'],
 ['right dg', 'right dg'],
 ['right dg', 'right ca1'],
 ['left dg', 'left ca1'],
 ['left dg', 'left dg'],
 ['left dg', 'left ca1'],
 ['left ca1', 'left dg'],
 ['left dg', 'left dg'],
 ['left dg', 'left ca1'],
 ['left ca1', 'left dg'],
 ['left dg', 'left dg'],
 ['left dg', 'left ca1'],
 ['left ca1', 'left dg'],
 ['left dg', 'left dg'],
 ['right dg', 'right dg'],
 ['right dg', 'right dg'],
 ['right dg', 'right ca1'],
 ['right dg', 'right dg'],
 ['right dg', 'right ca1'],
 ['left ca1', 'left dg'],
 ['left dg', 'left ca1'],
 ['left ca1', 'left dg'],
 ['left dg', 'left ca1'],
 ['left dg', 'left dg'],
 ['left dg', 'left dg'],
 ['left dg', 'left ca1'],
 ['left dg', 'left ca1'],
 ['left dg', 'left dg'],
 ['left dg', 'left dg'],
 ['left dg', 'left ca1'],
 ['left ca1', 'left dg'],
